**cisc-873-dm-w23-a3
Fake Reddit Prediction**
The dataset contains a collection of news articles that are labeled as reliable or unreliable.
The problem is to identify unreliable news articles. The input is a news article and the output is a binary classification of reliable or unreliable. The data mining function required is classification.
the input is text column and the output classification.
Some challenges include the fact that there are many different types of fake news and it can be difficult to identify them all. Additionally, some fake news articles may be intentionally misleading and difficult to detect.
The impact of fake news can be significant as it can lead to misinformation and confusion among readers. An ideal solution would be a system that can accurately identify unreliable news articles and prevent them from being shared widely.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cisc-873-dm-w23-a3/sample_submission.csv
/kaggle/input/cisc-873-dm-w23-a3/x_test.csv
/kaggle/input/cisc-873-dm-w23-a3/xy_train.csv


In [2]:
#import liberaries 
import re
import pickle
import sklearn
import holoviews as hv 
import nltk
from bokeh.io import output_notebook
output_notebook()
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from xgboost import XGBClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# some seeting for pandas and hvplot
pd.options.display.max_columns = 100 # Determine the maximum number of columns that I want to appear when displaying the dataframe
pd.options.display.max_rows = 300 # Determine the maximum number of rows that I want to appear when displaying the dataframe
pd.options.display.max_colwidth = 100 # Maximum width of columns
np.set_printoptions(threshold=2000)

Loading BokehJS ...

In [3]:
# Read training data (.csv) and inistialize in data_tr variable to use it
data_tr = pd.read_csv('/kaggle/input/cisc-873-dm-w23-a3/xy_train.csv')
data_tr.head()

,id,text,label
0,265723,"A group of friends began to volunteer at a homeless shelter after their neighbors protested. ""Se...",0
1,284269,"British Prime Minister @Theresa_May on Nerve Attack on Former Russian Spy: ""The government has c...",0
2,207715,"In 1961, Goodyear released a kit that allows PS2s to be brought to heel. https://m.youtube.com/w...",0
3,551106,"Happy Birthday, Bob Barker! The Price Is Right Host on How He'd Like to Be Remembered | ""As the ...",0
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Young Black Men Should Not be Dying Before Magic Jo...",0


In [4]:
print(type(data_tr))
print(data_tr.shape)
data_tr.head(5)

<class 'pandas.core.frame.DataFrame'>
(60000, 3)


,id,text,label
0,265723,"A group of friends began to volunteer at a homeless shelter after their neighbors protested. ""Se...",0
1,284269,"British Prime Minister @Theresa_May on Nerve Attack on Former Russian Spy: ""The government has c...",0
2,207715,"In 1961, Goodyear released a kit that allows PS2s to be brought to heel. https://m.youtube.com/w...",0
3,551106,"Happy Birthday, Bob Barker! The Price Is Right Host on How He'd Like to Be Remembered | ""As the ...",0
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Young Black Men Should Not be Dying Before Magic Jo...",0


In [5]:
# Read testing data (.csv) and inistialize in data_ts variable to use it
data_ts = pd.read_csv('/kaggle/input/cisc-873-dm-w23-a3/x_test.csv')
data_ts.head(5)

,id,text
0,0,stargazer
1,1,yeah
2,2,PD: Phoenix car thief gets instructions from YouTube video
3,3,"As Trump Accuses Iran, He Has One Problem: His Own Credibility"
4,4,"""Believers"" - Hezbollah 2011"


In [6]:
print(type(data_ts))
print(data_ts.shape)
data_ts.head(5)

<class 'pandas.core.frame.DataFrame'>
(59151, 2)


,id,text
0,0,stargazer
1,1,yeah
2,2,PD: Phoenix car thief gets instructions from YouTube video
3,3,"As Trump Accuses Iran, He Has One Problem: His Own Credibility"
4,4,"""Believers"" - Hezbollah 2011"


In [7]:
# Display the column's name in training and testing data
print(data_tr.columns)
print(data_ts.columns)

Index(['id', 'text', 'label'], dtype='object')
Index(['id', 'text'], dtype='object')


In [8]:
# Display the dataframe's information in training and testing data
print(data_tr.info())
print(data_ts.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      60000 non-null  int64 
 1   text    60000 non-null  object
 2   label   60000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59151 entries, 0 to 59150
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      59151 non-null  int64 
 1   text    59151 non-null  object
dtypes: int64(1), object(1)
memory usage: 924.4+ KB
None


In [9]:
nltk.download('punkt')
nltk.download('stopwords')

stemmer = SnowballStemmer("english")   # It is the method used to return the word to its original form
stop_words = set(stopwords.words("english")) # It is the method of producing a stop words

def clean_text(text):
    """ steps:
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm
    """
    # IGNORECASE : is a flag allows for case-insensitive matching of the Regular Expression with the given string
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE) # Remove any more than one space
    RE_TAGS = re.compile(r"<[^>]+>") # Remove web tags
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE) # Remove any leter does not english charachter
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE) # Remove any single character

    text = re.sub(RE_TAGS, " ", text)# Replace any tag with a single space.
    text = re.sub(RE_ASCII, " ", text) # Replace any non english character with a single space.
    text = re.sub(RE_SINGLECHAR, " ", text) # Replace any single character with a single space.
    text = re.sub(RE_WSPACE, " ", text)  # Replace any more than one space with a single space.

    word_tokens = word_tokenize(text) # split the sentence into words
    words_tokens_lower = [word.lower() for word in word_tokens] # Convert all letters to small letters

    # words_filtered (Words can be filtered based on how many times they appear)
    # stemmer used to return the word to its original form.
    words_filtered = [
        stemmer.stem(word) for word in words_tokens_lower if word not in stop_words
    ]

    # Join all words in text_clean and separate them by space.
    text_clean = " ".join(words_filtered)
    return text_clean

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# call clean_text function that take string as a parameter to test the function
clean_text("Python is a basic programming language .")

'python basic program languag'

In [11]:
# Clean texts by taking any text that's length is greater than 25
# map is an iterator function that returns a result after applying a clean_text function to each item in an iterable 
# lambda is a function used to apply certain functions to all rows of a data set.
# lambda take one argument (x) then put x in clean_text function
# if statement means ( if input x is string enter x to clean_text function then the result put in data['clean_com'] if not return x in data['clean_com'] as it is  )
data_tr['clean_text']=data_tr.loc[data_tr['text'].str.len()>25,"text"]
data_tr['clean_text']=data_tr['clean_text'].map(
    lambda x: clean_text(x) if isinstance(x, str) else x   
)

data_ts['clean_text']=data_ts.loc[data_ts['text'].str.len()>0,"text"]
data_ts['clean_text']=data_ts['clean_text'].map(
    lambda x: clean_text(x) if isinstance(x, str) else x   
)



In [12]:
data_tr

,id,text,label,clean_text
0,265723,"A group of friends began to volunteer at a homeless shelter after their neighbors protested. ""Se...",0,group friend began volunt homeless shelter neighbor protest see anoth person also need natur lik...
1,284269,"British Prime Minister @Theresa_May on Nerve Attack on Former Russian Spy: ""The government has c...",0,british prime minist theresa may nerv attack former russian spi govern conclud high like russia ...
2,207715,"In 1961, Goodyear released a kit that allows PS2s to be brought to heel. https://m.youtube.com/w...",0,goodyear releas kit allow ps brought heel https youtub com watch alxulk cg zwillc fish midatlant...
3,551106,"Happy Birthday, Bob Barker! The Price Is Right Host on How He'd Like to Be Remembered | ""As the ...",0,happi birthday bob barker price right host like rememb man said ave pet spay neuter fuckincorpor...
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Young Black Men Should Not be Dying Before Magic Jo...",0,obama nation innoc cop unarm young black men die magic johnson jimbobshawobodob olymp athlet sho...
...,...,...,...,...
59995,70046,"Finish Sniper Simo H盲yh盲 during the invasion of Finland by the USSR (1939, colorized)",0,finish sniper simo yh invas finland ussr color
59996,189377,"Nigerian Prince Scam took $110K from Kansas man; 10 years later, he's getting it back",1,nigerian princ scam took kansa man year later get back
59997,93486,Is It Safe To Smoke Marijuana During Pregnancy? You鈥檇 Be Surprised Of The Answer | no,0,safe smoke marijuana pregnanc surpris answer
59998,140950,Julius Caesar upon realizing that everyone in the room has a knife except him (44 bc),0,julius caesar upon realiz everyon room knife except bc


In [13]:
data_ts

,id,text,clean_text
0,0,stargazer,stargaz
1,1,yeah,yeah
2,2,PD: Phoenix car thief gets instructions from YouTube video,pd phoenix car thief get instruct youtub video
3,3,"As Trump Accuses Iran, He Has One Problem: His Own Credibility",trump accus iran one problem credibl
4,4,"""Believers"" - Hezbollah 2011",believ hezbollah
...,...,...,...
59146,59146,Bicycle taxi drivers of New Delhi,bicycl taxi driver new delhi
59147,59147,Trump blows up GOP's formula for winning House races,trump blow gop formula win hous race
59148,59148,"Napoleon returns from his exile on the island of Elba. (March 1815), Colourised",napoleon return exil island elba march colouris
59149,59149,Deep down he always wanted to be a ballet dancer,deep alway want ballet dancer


In [14]:
# Examine the label column for unique values and the number of times they appear.
data_tr['label'].value_counts()

0    32172
1    27596
2      232
Name: label, dtype: int64

In [15]:
# drop rows that has label = 2 
drop2=data_tr['label']==2
data_tr.drop(data_tr.index[drop2],inplace=True)

In [16]:
# Examine the label column for unique values and the number of times they appear.
data_tr['label'].value_counts()

0    32172
1    27596
Name: label, dtype: int64

In [17]:
# copy data in data_clean
data_tr_clean=data_tr.copy()

In [18]:
data_tr_clean.head(5)

,id,text,label,clean_text
0,265723,"A group of friends began to volunteer at a homeless shelter after their neighbors protested. ""Se...",0,group friend began volunt homeless shelter neighbor protest see anoth person also need natur lik...
1,284269,"British Prime Minister @Theresa_May on Nerve Attack on Former Russian Spy: ""The government has c...",0,british prime minist theresa may nerv attack former russian spi govern conclud high like russia ...
2,207715,"In 1961, Goodyear released a kit that allows PS2s to be brought to heel. https://m.youtube.com/w...",0,goodyear releas kit allow ps brought heel https youtub com watch alxulk cg zwillc fish midatlant...
3,551106,"Happy Birthday, Bob Barker! The Price Is Right Host on How He'd Like to Be Remembered | ""As the ...",0,happi birthday bob barker price right host like rememb man said ave pet spay neuter fuckincorpor...
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Young Black Men Should Not be Dying Before Magic Jo...",0,obama nation innoc cop unarm young black men die magic johnson jimbobshawobodob olymp athlet sho...


In [19]:
# Word Frequency of most common words
# Split all words with whitespace between them, then put them in word_freq.
word_freq_tr = pd.Series(" ".join(data_tr_clean["clean_text"]).split()).value_counts()
word_freq_tr[1:40] # display the first 40 words with frequencies

one         3285
like        3128
new         2998
look        2847
color       2737
man         2729
get         2602
trump       2578
say         2347
peopl       2316
use         2307
first       2248
make        2227
old         2226
time        2027
poster      2000
found       1999
day         1935
war         1858
post        1648
world       1570
work        1531
show        1513
us          1506
american    1504
take        1491
life        1482
psbattl     1470
help        1442
go          1420
state       1409
back        1369
two         1364
school      1345
see         1329
photo       1324
made        1314
right       1311
save        1308
dtype: int64

In [20]:
# display list most uncommon words  
# reset_index is reset the index of the DataFrame
word_freq_tr[-10:].reset_index(name="freq")

,index,freq
0,angriff,1
1,delusion,1
2,wane,1
3,undament,1
4,miku,1
5,hatsun,1
6,nfler,1
7,hicock,1
8,mccall,1
9,wahr,1


In [21]:
# Distribution of ratings
data_tr_clean["label"].value_counts(normalize=True) # count proportions of label

0    0.538281
1    0.461719
Name: label, dtype: float64

In [22]:
"""
Compute unique word vector with frequencies
exclude very uncommon (<10 obsv.) and common (>=30%) words
use pairs of two words (ngram)
"""
# TfidfVectorizer convert a collection of raw documents to a matrix of TF-IDF features.
# min_df ignore terms that have a document frequency strictly lower than the given threshold
# max_df ignore terms that have a document frequency strictly higher than the given threshold
# ngram_range Two words have a higher correlation than the threshold and frequently appear together.
# ngram_range=(a,b)-> a is the minimum and b is the maximum size of ngrams

vectorizer = TfidfVectorizer(
    max_df=0.3, min_df=10, ngram_range=(1, 2)
)
vectorizer.fit(data_tr_clean["clean_text"])

# vectorizer = TfidfVectorizer(
#     analyzer="word", max_df=0.3, min_df=10, ngram_range=(1, 2), norm="l2"
# )
# vectorizer.fit(data_clean["clean_text"])

TfidfVectorizer(max_df=0.3, min_df=10, ngram_range=(1, 2))

In [23]:
# Vector representation of vocabulary show some sample from our data_clean
word_vector = pd.Series(vectorizer.vocabulary_).sample(5, random_state=1) # By sample, I choose the number of vocabulary that I want to display.
print(f"Unique word (ngram) vector extract:\n\n {word_vector}")

Unique word (ngram) vector extract:

 eli       2666
go far    3595
bamboo     650
wisdom    9837
pocket    6705
dtype: int64


In [24]:
data_tr_clean

,id,text,label,clean_text
0,265723,"A group of friends began to volunteer at a homeless shelter after their neighbors protested. ""Se...",0,group friend began volunt homeless shelter neighbor protest see anoth person also need natur lik...
1,284269,"British Prime Minister @Theresa_May on Nerve Attack on Former Russian Spy: ""The government has c...",0,british prime minist theresa may nerv attack former russian spi govern conclud high like russia ...
2,207715,"In 1961, Goodyear released a kit that allows PS2s to be brought to heel. https://m.youtube.com/w...",0,goodyear releas kit allow ps brought heel https youtub com watch alxulk cg zwillc fish midatlant...
3,551106,"Happy Birthday, Bob Barker! The Price Is Right Host on How He'd Like to Be Remembered | ""As the ...",0,happi birthday bob barker price right host like rememb man said ave pet spay neuter fuckincorpor...
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Young Black Men Should Not be Dying Before Magic Jo...",0,obama nation innoc cop unarm young black men die magic johnson jimbobshawobodob olymp athlet sho...
...,...,...,...,...
59995,70046,"Finish Sniper Simo H盲yh盲 during the invasion of Finland by the USSR (1939, colorized)",0,finish sniper simo yh invas finland ussr color
59996,189377,"Nigerian Prince Scam took $110K from Kansas man; 10 years later, he's getting it back",1,nigerian princ scam took kansa man year later get back
59997,93486,Is It Safe To Smoke Marijuana During Pregnancy? You鈥檇 Be Surprised Of The Answer | no,0,safe smoke marijuana pregnanc surpris answer
59998,140950,Julius Caesar upon realizing that everyone in the room has a knife except him (44 bc),0,julius caesar upon realiz everyon room knife except bc


In [25]:
# splitting Trainig data into X_train and y_train
X=data_tr_clean['clean_text'] # X contains only clean_text column
y=data_tr_clean['label']   # y contains only label column
X_test=data_ts['clean_text'] # X_test contains only clean_text column

In [26]:
# transform each sentence to numeric vector with tf-idf value as elements
X_train_vec = vectorizer.transform(X)
X_test_vec = vectorizer.transform(X_test)
X_train_vec.get_shape()

(59768, 10061)

In [27]:
# Compare original comment text with its numeric vector representation
print(f"Original sentence:\n{X[3:4].values}\n")
# Feature is a dataframe that takes X_train_vec and converts it to an array, and the column is the name of the feature
features = pd.DataFrame(
    X_train_vec[3:4].toarray(), columns=vectorizer.get_feature_names()
)
nonempty_feat = features.loc[:, (features != 0).any(axis=0)]
print(f"Vector representation of sentence:\n {nonempty_feat}")

Original sentence:
['happi birthday bob barker price right host like rememb man said ave pet spay neuter fuckincorporateshil irish rover jolli rove tar imgur com true dh bqw https extern preview redd qseokdslwivxzdehgxxbceuxh vhemi muoxw yaqq jpg width crop smart auto webp eda ba bf ed fakealbumcov irish rover jolli rove tar kna fire made becam green place redd true cd anx https preview redd nw nexjtfaa jpg width crop smart auto webp ee da ce da mildlyinterest fire made becam green place rhym jarat true cf qh http imgur com crvur jpg wvf psbattl artwork jarat dagnummmong stop redd true qbhc https preview redd lsvmqhtjcz jpg width crop smart auto webp bfb dd ccafa cad fakealbumcov stop top today ss without jupit probabl asteroid impact artist use one vermont senat top surrog year stormtroop finish anim imgur com true cal https extern preview redd vwnj gq tvah fdc ahmc qyzjj sw fqlrwqauk jpg width crop smart auto webp fba de ac subredditsimul without jupit probabl asteroid impact artist 

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


##### Validation set

In [28]:
from sklearn.model_selection import PredefinedSplit
# Further split the original training set to a train and a validation set
X_train, X_val, y_train, y_val = train_test_split(
    X, y, train_size = 0.8, stratify = y, random_state = 2022)

# Create a list where train data indices are -1 and validation data indices are 0
# X_train2 (new training set), X_train
split_index = [-1 if x in X_train.index else 0 for x in X.index]

# Use the list to create PredefinedSplit
pds = PredefinedSplit(test_fold = split_index)


In [29]:
# Built pipline
# The pipeline's goal is to combine numerous processes that can be cross-validated while modifying various parameters.
# It does this by allowing set parameters for each step using their names and parameter names separated by a "__"
# It takes steps as a prameter that contain all the preprocessing, vectorization, and normalization that I need.
# It saves time by applying any preprocessing to both train and test data without repeating the process.

pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="word", norm="l2")), ("my_classifier",  MLPClassifier(
        random_state=1,
        solver="adam",
        hidden_layer_sizes=(12, 12, 12),
        activation="relu",
        early_stopping=True,
        n_iter_no_change=1,
    ))])

# define parameter space to test
params = {
    "tfidf__ngram_range": [(1, 2), (1, 3)],
    # tfidf__ngram_range points to TfidfVectorizer -> ngram_range
    # ngram_range Two words have a higher correlation than the threshold and frequently appear together.
    # ngram_range=(a,b)-> a is the minimum and b is the maximum size of ngrams
    "tfidf__max_df": np.arange(0.3, 0.8),
    # max_df ignore terms that have a document frequency strictly higher than the given threshold
    "tfidf__min_df": np.arange(5, 100),
    # min_df ignore terms that have a document frequency strictly lower than the given threshold
}

pipe_clf = RandomizedSearchCV(
    pipe, params, cv=pds, verbose=1, n_jobs=2, 
    scoring='roc_auc')

# here we still use X but the Randomized search model by use validation set 
# fit the pipeline
pipe_clf.fit(X, y)

print('best score {}'.format(pipe_clf.best_score_))
print('best score {}'.format(pipe_clf.best_params_))

Fitting 1 folds for each of 10 candidates, totalling 10 fits
best score 0.8605539695954626
best score {'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 13, 'tfidf__max_df': 0.3}


In [30]:
# run pipe with optimized parameters
best_params = pipe_clf.best_params_
pipe.set_params(**best_params).fit(X, y)
pipe_pred = pipe.predict(X_test)

In [31]:
# Use this cell to write the result in the excel sheet.
submission = pd.DataFrame()

submission['id'] = data_ts['id']

submission['label'] = pipe.predict_proba(X_test)[:,1]

submission.to_csv('sample_submission_walkthrough1.csv', index=False)

In [32]:
# Built pipline
# The pipeline's goal is to combine numerous processes that can be cross-validated while modifying various parameters.
# It does this by allowing set parameters for each step using their names and parameter names separated by a "__"
# It takes steps as a prameter that contain all the preprocessing, vectorization, and normalization that I need.
# It saves time by applying any preprocessing to both train and test data without repeating the process.

pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="word", norm="l2")), ("my_classifier",  MLPClassifier(
        random_state=1,
        solver="adam",
        hidden_layer_sizes=(12, 12, 12),
        activation="relu",
        early_stopping=True,
        n_iter_no_change=1,
    ))])

# define parameter space to test
params = {
    # ngram_range=(a,b)-> a is the minimum and b is the maximum size of ngrams
    "tfidf__max_df": np.arange(0.3, 0.8),
    # max_df ignore terms that have a document frequency strictly higher than the given threshold
    "tfidf__min_df": np.arange(5, 100),
    # min_df ignore terms that have a document frequency strictly lower than the given threshold
}

pipe_clf = BayesSearchCV(
    pipe, params, cv=pds, verbose=1, n_jobs=2, 
    scoring='roc_auc')

# here we still use X but the Bayes search model by use validation set 
# fit the pipeline
pipe_clf.fit(X, y)

print('best score {}'.format(pipe_clf.best_score_))
print('best score {}'.format(pipe_clf.best_params_))

Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fi

/opt/conda/lib/python3.7/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits


/opt/conda/lib/python3.7/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits


/opt/conda/lib/python3.7/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fi

In [33]:
# run pipe with optimized parameters
best_params = pipe_clf.best_params_
pipe.set_params(**best_params).fit(X, y)
pipe_pred = pipe.predict(X_test)

In [34]:
# Use this cell to write the result in the excel sheet.
submission = pd.DataFrame()

submission['id'] = data_ts['id']

submission['label'] = pipe.predict_proba(X_test)[:,1]

submission.to_csv('sample_submission_walkthrough2.csv', index=False)

In [35]:
# Built pipline
# The pipeline's goal is to combine numerous processes that can be cross-validated while modifying various parameters.
# It does this by allowing set parameters for each step using their names and parameter names separated by a "__"
# It takes steps as a prameter that contain all the preprocessing, vectorization, and normalization that I need.
# It saves time by applying any preprocessing to both train and test data without repeating the process.

pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="word", norm="l2")), ("my_classifier",  MLPClassifier(
        random_state=1,
        solver='lbfgs',
        hidden_layer_sizes=(12, 12, 12),
        activation="relu",
        early_stopping=True,
        alpha=0.0001,
        learning_rate= 'adaptive',
    ))])

# define parameter space to test
params = {
    "tfidf__ngram_range": [(1, 2), (1, 3)],
    # tfidf__ngram_range points to TfidfVectorizer -> ngram_range
    # ngram_range Two words have a higher correlation than the threshold and frequently appear together.
    # ngram_range=(a,b)-> a is the minimum and b is the maximum size of ngrams
    "tfidf__max_df": np.arange(0.3, 0.8),
    # max_df ignore terms that have a document frequency strictly higher than the given threshold
    "tfidf__min_df": np.arange(5, 100),
    # min_df ignore terms that have a document frequency strictly lower than the given threshold
}

pipe_clf = RandomizedSearchCV(
    pipe, params, cv=pds, verbose=1, n_jobs=2, 
    scoring='roc_auc')

# here we still use X but the Randomized search model by use validation set 
# fit the pipeline
pipe_clf.fit(X, y)

print('best score {}'.format(pipe_clf.best_score_))
print('best score {}'.format(pipe_clf.best_params_))

Fitting 1 folds for each of 10 candidates, totalling 10 fits
best score 0.8598078855371842
best score {'tfidf__ngram_range': (1, 3), 'tfidf__min_df': 17, 'tfidf__max_df': 0.3}


/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [36]:
# run pipe with optimized parameters
best_params = pipe_clf.best_params_
pipe.set_params(**best_params).fit(X, y)
pipe_pred = pipe.predict(X_test)

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [37]:
# Use this cell to write the result in the excel sheet.
submission = pd.DataFrame()

submission['id'] = data_ts['id']

submission['label'] = pipe.predict_proba(X_test)[:,1]

submission.to_csv('sample_submission_walkthrough4.csv', index=False)

In [38]:
# Built pipline
# The pipeline's goal is to combine numerous processes that can be cross-validated while modifying various parameters.
# It does this by allowing set parameters for each step using their names and parameter names separated by a "__"
# It takes steps as a prameter that contain all the preprocessing, vectorization, and normalization that I need.
# It saves time by applying any preprocessing to both train and test data without repeating the process.

pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="word", norm="l2")), ("my_classifier",  XGBClassifier())])

# define parameter space to test
params = {
    "tfidf__ngram_range": [(1, 2), (1, 3)],
    # tfidf__ngram_range points to TfidfVectorizer -> ngram_range
    # ngram_range Two words have a higher correlation than the threshold and frequently appear together.
    # ngram_range=(a,b)-> a is the minimum and b is the maximum size of ngrams
    "tfidf__max_df": np.arange(0.3, 0.8),
    # max_df ignore terms that have a document frequency strictly higher than the given threshold
    "tfidf__min_df": np.arange(5, 100),
    # min_df ignore terms that have a document frequency strictly lower than the given threshold
    'my_classifier__min_child_weight': [20,40,80],
    # min_child_weight is a minimal total of the weights in a child.
    'my_classifier__max_depth':[50,60,70],  
    # max_depth is a maximum depth of a tree
    'my_classifier__gamma':[0.5, 1, 1.5, 2, 5],
    # gamma is a minimum loss that we need it to split tree
    'my_classifier__colsample_bytree':[0.6, 0.8, 1.0]
}

pipe_clf = RandomizedSearchCV(
    pipe, params, cv=pds, verbose=1, n_jobs=2, 
    scoring='roc_auc')

# here we still use X but the Randomized search model by use validation set 
# fit the pipeline
pipe_clf.fit(X, y)

print('best score {}'.format(pipe_clf.best_score_))
print('best score {}'.format(pipe_clf.best_params_))

Fitting 1 folds for each of 10 candidates, totalling 10 fits
best score 0.8283922616410385
best score {'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 32, 'tfidf__max_df': 0.3, 'my_classifier__min_child_weight': 20, 'my_classifier__max_depth': 50, 'my_classifier__gamma': 1, 'my_classifier__colsample_bytree': 1.0}


In [39]:
# run pipe with optimized parameters
best_params = pipe_clf.best_params_
pipe.set_params(**best_params).fit(X, y)
pipe_pred = pipe.predict(X_test)

In [40]:
# Use this cell to write the result in the excel sheet.
submission = pd.DataFrame()

submission['id'] = data_ts['id']

submission['label'] = pipe.predict_proba(X_test)[:,1]

submission.to_csv('sample_submission_walkthrough5.csv', index=False)

In [41]:
# # Built pipline
# # The pipeline's goal is to combine numerous processes that can be cross-validated while modifying various parameters.
# # It does this by allowing set parameters for each step using their names and parameter names separated by a "__"
# # It takes steps as a prameter that contain all the preprocessing, vectorization, and normalization that I need.
# # It saves time by applying any preprocessing to both train and test data without repeating the process.

# pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="word", norm="l2")), ("my_classifier",  XGBClassifier())])

# # define parameter space to test
# params = {
#     "tfidf__ngram_range": [(1, 2), (1, 3)],
#     # tfidf__ngram_range points to TfidfVectorizer -> ngram_range
#     # ngram_range Two words have a higher correlation than the threshold and frequently appear together.
#     # ngram_range=(a,b)-> a is the minimum and b is the maximum size of ngrams
#     "tfidf__max_df": np.arange(0.3, 0.8),
#     # max_df ignore terms that have a document frequency strictly higher than the given threshold
#     "tfidf__min_df": np.arange(5, 100),
#     # min_df ignore terms that have a document frequency strictly lower than the given threshold
#     'my_classifier__min_child_weight': [20,40,80],
#     # min_child_weight is a minimal total of the weights in a child.
#     'my_classifier__max_depth':[50,60,70],  
#     # max_depth is a maximum depth of a tree
#     'my_classifier__gamma':[0.5, 1, 1.5, 2, 5],
#     # gamma is a minimum loss that we need it to split tree
#     'my_classifier__colsample_bytree':[0.6, 0.8, 1.0]
# }

# pipe_clf = GridSearchCV(
#     pipe, params, cv=pds, verbose=1, n_jobs=2, 
#     scoring='roc_auc')

# # here we still use X but the Randomized search model by use validation set 
# # fit the pipeline
# pipe_clf.fit(X, y)

# print('best score {}'.format(pipe_clf.best_score_))
# print('best score {}'.format(pipe_clf.best_params_))

In [42]:
# # run pipe with optimized parameters
# best_params = pipe_clf.best_params_
# pipe.set_params(**best_params).fit(X, y)
# pipe_pred = pipe.predict(X_test)

In [43]:
# # Use this cell to write the result in the excel sheet.
# submission = pd.DataFrame()

# submission['id'] = data_ts['id']

# submission['label'] = pipe.predict_proba(X_test)[:,1]

# submission.to_csv('sample_submission_walkthrough10.csv', index=False)

In [44]:
# # Built pipline
# # The pipeline's goal is to combine numerous processes that can be cross-validated while modifying various parameters.
# # It does this by allowing set parameters for each step using their names and parameter names separated by a "__"
# # It takes steps as a prameter that contain all the preprocessing, vectorization, and normalization that I need.
# # It saves time by applying any preprocessing to both train and test data without repeating the process.

# pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="word", norm="l2")), ("my_classifier",  XGBClassifier())])

# # define parameter space to test
# params = {
#     "tfidf__ngram_range": [(1, 2), (1, 3)],
#     # tfidf__ngram_range points to TfidfVectorizer -> ngram_range
#     # ngram_range Two words have a higher correlation than the threshold and frequently appear together.
#     # ngram_range=(a,b)-> a is the minimum and b is the maximum size of ngrams
#     "tfidf__max_df": np.arange(0.3, 0.8),
#     # max_df ignore terms that have a document frequency strictly higher than the given threshold
#     "tfidf__min_df": np.arange(5, 100),
#     # min_df ignore terms that have a document frequency strictly lower than the given threshold
#     'my_classifier__min_child_weight': [20,40,80],
#     # min_child_weight is a minimal total of the weights in a child.
#     'my_classifier__max_depth':[50,60,70],  
#     # max_depth is a maximum depth of a tree
#     'my_classifier__gamma':[0.5, 1, 1.5, 2, 5],
#     # gamma is a minimum loss that we need it to split tree
#     'my_classifier__colsample_bytree':[0.6, 0.8, 1.0]
# }

# pipe_clf = BayesSearchCV(
#     pipe, params, cv=pds, verbose=1, n_jobs=2, 
#     scoring='roc_auc')

# # here we still use X but the Randomized search model by use validation set 
# # fit the pipeline
# pipe_clf.fit(X, y)

# print('best score {}'.format(pipe_clf.best_score_))
# print('best score {}'.format(pipe_clf.best_params_))

In [45]:
# # run pipe with optimized parameters
# best_params = pipe_clf.best_params_
# pipe.set_params(**best_params).fit(X, y)
# pipe_pred = pipe.predict(X_test)

In [46]:
# # Use this cell to write the result in the excel sheet.
# submission = pd.DataFrame()

# submission['id'] = data_ts['id']

# submission['label'] = pipe.predict_proba(X_test)[:,1]

# submission.to_csv('sample_submission_walkthrough11.csv', index=False)

In [47]:
# # Built pipline
# # The pipeline's goal is to combine numerous processes that can be cross-validated while modifying various parameters.
# # It does this by allowing set parameters for each step using their names and parameter names separated by a "__"
# # It takes steps as a prameter that contain all the preprocessing, vectorization, and normalization that I need.
# # It saves time by applying any preprocessing to both train and test data without repeating the process.

# pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="word", norm="l2")), ("my_classifier",  LogisticRegression())])

# # define parameter space to test
# params = {
#     "tfidf__ngram_range": [(1, 2), (1, 3)],
#     # tfidf__ngram_range points to TfidfVectorizer -> ngram_range
#     # ngram_range Two words have a higher correlation than the threshold and frequently appear together.
#     # ngram_range=(a,b)-> a is the minimum and b is the maximum size of ngrams
#     "tfidf__max_df": np.arange(0.3, 0.8),
#     # max_df ignore terms that have a document frequency strictly higher than the given threshold
#     "tfidf__min_df": np.arange(5, 100),
#     # min_df ignore terms that have a document frequency strictly lower than the given threshold
#     'my_classifier__C': [100],
#     # C controls the penality strength 
#     # my_classifier__C points to my_classifier->C
#     'my_classifier__max_iter':[100], 
#     # max_iter is a maximum number of iterations
#     # my_classifier__max_iter points to my_classifier-> max_iter
#     'my_classifier__tol':[1e-4,1e-5,1e-3]
#     # tol is a tolerance for stopping
#     # my_classifier__tol points to my_classifier-> tol

# }

# pipe_clf = GridSearchCV(
#     pipe, params, cv=pds, verbose=1, n_jobs=2, 
#     scoring='roc_auc')

# # here we still use X but the Grid search model by use validation set 
# # fit the pipeline
# pipe_clf.fit(X, y)

# print('best score {}'.format(pipe_clf.best_score_))
# print('best score {}'.format(pipe_clf.best_params_))

In [48]:
# # run pipe with optimized parameters
# best_params = pipe_clf.best_params_
# pipe.set_params(**best_params).fit(X, y)
# pipe_pred = pipe.predict(X_test)

In [49]:
# # Use this cell to write the result in the excel sheet.
# submission = pd.DataFrame()

# submission['id'] = data_ts['id']

# submission['label'] = pipe.predict_proba(X_test)[:,1]

# submission.to_csv('sample_submission_walkthrough6.csv', index=False)

In [50]:
# Built pipline
# The pipeline's goal is to combine numerous processes that can be cross-validated while modifying various parameters.
# It does this by allowing set parameters for each step using their names and parameter names separated by a "__"
# It takes steps as a prameter that contain all the preprocessing, vectorization, and normalization that I need.
# It saves time by applying any preprocessing to both train and test data without repeating the process.

pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="word", norm="l2")), ("my_classifier",  LogisticRegression())])

# define parameter space to test
params = {
    "tfidf__max_df": np.arange(0.3, 0.8),
    # max_df ignore terms that have a document frequency strictly higher than the given threshold
    "tfidf__min_df": np.arange(5, 100),
    # min_df ignore terms that have a document frequency strictly lower than the given threshold
    'my_classifier__C': [100,200,300],
    # C controls the penality strength 
    # my_classifier__C points to my_classifier->C
    'my_classifier__max_iter':[100 ,200, 300], 
    # max_iter is a maximum number of iterations
    # my_classifier__max_iter points to my_classifier-> max_iter
    'my_classifier__tol':[1e-4,1e-5,1e-3]
    # tol is a tolerance for stopping
    # my_classifier__tol points to my_classifier-> tol

}

pipe_clf = BayesSearchCV(
    pipe, params, cv=pds, verbose=1, n_jobs=2, 
    scoring='roc_auc')

# here we still use X but the Randomized search model by use validation set 
# fit the pipeline
pipe_clf.fit(X, y)

print('best score {}'.format(pipe_clf.best_score_))
print('best score {}'.format(pipe_clf.best_params_))

Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fi

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REAC

Fitting 1 folds for each of 1 candidates, totalling 1 fits
best score 0.8497117748069006
best score OrderedDict([('my_classifier__C', 100), ('my_classifier__max_iter', 300), ('my_classifier__tol', 0.001), ('tfidf__max_df', 0.3), ('tfidf__min_df', 22)])


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [51]:
# run pipe with optimized parameters
best_params = pipe_clf.best_params_
pipe.set_params(**best_params).fit(X, y)
pipe_pred = pipe.predict(X_test)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [52]:
# Use this cell to write the result in the excel sheet.
submission = pd.DataFrame()

submission['id'] = data_ts['id']

submission['label'] = pipe.predict_proba(X_test)[:,1]

submission.to_csv('sample_submission_walkthrough7.csv', index=False)

In [53]:
# Built pipline
# The pipeline's goal is to combine numerous processes that can be cross-validated while modifying various parameters.
# It does this by allowing set parameters for each step using their names and parameter names separated by a "__"
# It takes steps as a prameter that contain all the preprocessing, vectorization, and normalization that I need.
# It saves time by applying any preprocessing to both train and test data without repeating the process.

pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="word", norm="l2")), ("my_classifier",  RandomForestClassifier())])

# define parameter space to test
params = {
    "tfidf__ngram_range": [(1, 2), (1, 3)],
    # tfidf__ngram_range points to TfidfVectorizer -> ngram_range
    # ngram_range Two words have a higher correlation than the threshold and frequently appear together.
    # ngram_range=(a,b)-> a is the minimum and b is the maximum size of ngrams
    "tfidf__max_df": np.arange(0.3, 0.8),
    # max_df ignore terms that have a document frequency strictly higher than the given threshold
    "tfidf__min_df": np.arange(5, 100),
    # min_df ignore terms that have a document frequency strictly lower than the given threshold
    'my_classifier__n_estimators': [170,200,250],
    # n_estimators is a number of trees 
    # my_classifier__n_estimators points to my_classifier->n_estimators
    'my_classifier__max_depth':[70,80,90],   
    # max_depth is a maximum depth of the tree
    # my_classifier__max_depth points to my_classifier-> max_depth
    'my_classifier__max_features':[10,20,30]
    # max_features is a maximum number of features
    # my_classifier__max_features points to my_classifier-> max_features

}

pipe_clf = RandomizedSearchCV(
    pipe, params, cv=pds, verbose=1, n_jobs=2, 
    scoring='roc_auc')

# here we still use X but the Randomized search model by use validation set 
# fit the pipeline
pipe_clf.fit(X, y)

print('best score {}'.format(pipe_clf.best_score_))
print('best score {}'.format(pipe_clf.best_params_))

Fitting 1 folds for each of 10 candidates, totalling 10 fits
best score 0.8370142953219597
best score {'tfidf__ngram_range': (1, 3), 'tfidf__min_df': 17, 'tfidf__max_df': 0.3, 'my_classifier__n_estimators': 170, 'my_classifier__max_features': 20, 'my_classifier__max_depth': 90}


In [54]:
# run pipe with optimized parameters
best_params = pipe_clf.best_params_
pipe.set_params(**best_params).fit(X, y)
pipe_pred = pipe.predict(X_test)

In [55]:
# Use this cell to write the result in the excel sheet.
submission = pd.DataFrame()

submission['id'] = data_ts['id']

submission['label'] = pipe.predict_proba(X_test)[:,1]

submission.to_csv('sample_submission_walkthrough8.csv', index=False)

In [56]:
# pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="word", norm="l2")), ("my_classifier",  RandomForestClassifier())])

# # define parameter space to test
# params = {
#     "tfidf__ngram_range": [(1, 2), (1, 3)],
#     # tfidf__ngram_range points to TfidfVectorizer -> ngram_range
#     # ngram_range Two words have a higher correlation than the threshold and frequently appear together.
#     # ngram_range=(a,b)-> a is the minimum and b is the maximum size of ngrams
#     "tfidf__max_df": np.arange(0.3, 0.8),
#     # max_df ignore terms that have a document frequency strictly higher than the given threshold
#     "tfidf__min_df": np.arange(5, 100),
#     # min_df ignore terms that have a document frequency strictly lower than the given threshold
#     'my_classifier__n_estimators': [170,200,250],
#     # n_estimators is a number of trees 
#     # my_classifier__n_estimators points to my_classifier->n_estimators
#     'my_classifier__max_depth':[70,80,90],   
#     # max_depth is a maximum depth of the tree
#     # my_classifier__max_depth points to my_classifier-> max_depth
#     'my_classifier__max_features':[10,20,30]
#     # max_features is a maximum number of features
#     # my_classifier__max_features points to my_classifier-> max_features

# }

# pipe_clf = GridSearchCV(
#     pipe, params, cv=pds, verbose=1, n_jobs=2, 
#     scoring='roc_auc')

# # here we still use X but the Randomized search model by use validation set 
# # fit the pipeline
# pipe_clf.fit(X, y)

# print('best score {}'.format(pipe_clf.best_score_))
# print('best score {}'.format(pipe_clf.best_params_))

In [57]:
# # run pipe with optimized parameters
# best_params = pipe_clf.best_params_
# pipe.set_params(**best_params).fit(X, y)
# pipe_pred = pipe.predict(X_test)

In [58]:
# # Use this cell to write the result in the excel sheet.
# submission = pd.DataFrame()

# submission['id'] = data_ts['id']

# submission['label'] = pipe.predict_proba(X_test)[:,1]

# submission.to_csv('sample_submission_walkthrough9.csv', index=False)

In [59]:
# Built pipline
# The pipeline's goal is to combine numerous processes that can be cross-validated while modifying various parameters.
# It does this by allowing set parameters for each step using their names and parameter names separated by a "__"
# It takes steps as a prameter that contain all the preprocessing, vectorization, and normalization that I need.
# It saves time by applying any preprocessing to both train and test data without repeating the process.

pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="char", norm="l2")), ("my_classifier",  XGBClassifier())])

# define parameter space to test
params = {
    "tfidf__ngram_range": [(1, 2), (1, 3)],
    # tfidf__ngram_range points to TfidfVectorizer -> ngram_range
    # ngram_range Two words have a higher correlation than the threshold and frequently appear together.
    # ngram_range=(a,b)-> a is the minimum and b is the maximum size of ngrams
    "tfidf__max_df": np.arange(0.3, 0.8),
    # max_df ignore terms that have a document frequency strictly higher than the given threshold
    "tfidf__min_df": np.arange(5, 100),
    # min_df ignore terms that have a document frequency strictly lower than the given threshold
    'my_classifier__min_child_weight': [20,40,80],
    # min_child_weight is a minimal total of the weights in a child.
    'my_classifier__max_depth':[50,60,70],  
    # max_depth is a maximum depth of a tree
    'my_classifier__gamma':[0.5, 1, 1.5, 2, 5],
    # gamma is a minimum loss that we need it to split tree
    'my_classifier__colsample_bytree':[0.6, 0.8, 1.0]
}

pipe_clf = RandomizedSearchCV(
    pipe, params, cv=pds, verbose=1, n_jobs=2, 
    scoring='roc_auc')

# here we still use X but the Randomized search model by use validation set 
# fit the pipeline
pipe_clf.fit(X, y)

print('best score {}'.format(pipe_clf.best_score_))
print('best score {}'.format(pipe_clf.best_params_))

Fitting 1 folds for each of 10 candidates, totalling 10 fits


/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REAC

best score 0.8243818310497056
best score {'tfidf__ngram_range': (1, 3), 'tfidf__min_df': 59, 'tfidf__max_df': 0.3, 'my_classifier__min_child_weight': 20, 'my_classifier__max_depth': 70, 'my_classifier__gamma': 2, 'my_classifier__colsample_bytree': 1.0}


In [60]:
# run pipe with optimized parameters
best_params = pipe_clf.best_params_
pipe.set_params(**best_params).fit(X, y)
pipe_pred = pipe.predict(X_test)

In [61]:
# Use this cell to write the result in the excel sheet.
submission = pd.DataFrame()

submission['id'] = data_ts['id']

submission['label'] = pipe.predict_proba(X_test)[:,1]

submission.to_csv('sample_submission_walkthrough12.csv', index=False)

In [62]:
# # Built pipline
# # The pipeline's goal is to combine numerous processes that can be cross-validated while modifying various parameters.
# # It does this by allowing set parameters for each step using their names and parameter names separated by a "__"
# # It takes steps as a prameter that contain all the preprocessing, vectorization, and normalization that I need.
# # It saves time by applying any preprocessing to both train and test data without repeating the process.

# pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="char", norm="l2")), ("my_classifier",  XGBClassifier())])

# # define parameter space to test
# params = {
#     "tfidf__ngram_range": [(1, 2), (1, 3)],
#     # tfidf__ngram_range points to TfidfVectorizer -> ngram_range
#     # ngram_range Two words have a higher correlation than the threshold and frequently appear together.
#     # ngram_range=(a,b)-> a is the minimum and b is the maximum size of ngrams
#     "tfidf__max_df": np.arange(0.3, 0.8),
#     # max_df ignore terms that have a document frequency strictly higher than the given threshold
#     "tfidf__min_df": np.arange(5, 100),
#     # min_df ignore terms that have a document frequency strictly lower than the given threshold
#     'my_classifier__min_child_weight': [20,40,80],
#     # min_child_weight is a minimal total of the weights in a child.
#     'my_classifier__max_depth':[50,60,70],  
#     # max_depth is a maximum depth of a tree
#     'my_classifier__gamma':[0.5, 1, 1.5, 2, 5],
#     # gamma is a minimum loss that we need it to split tree
#     'my_classifier__colsample_bytree':[0.6, 0.8, 1.0]
# }

# pipe_clf = GridSearchCV(
#     pipe, params, cv=pds, verbose=1, n_jobs=2, 
#     scoring='roc_auc')

# # here we still use X but the Randomized search model by use validation set 
# # fit the pipeline
# pipe_clf.fit(X, y)

# print('best score {}'.format(pipe_clf.best_score_))
# print('best score {}'.format(pipe_clf.best_params_))

In [63]:
# # run pipe with optimized parameters
# best_params = pipe_clf.best_params_
# pipe.set_params(**best_params).fit(X, y)
# pipe_pred = pipe.predict(X_test)

In [64]:
# # Use this cell to write the result in the excel sheet.
# submission = pd.DataFrame()

# submission['id'] = data_ts['id']

# submission['label'] = pipe_clf.predict_proba(X_test)[:,1]

# submission.to_csv('sample_submission_walkthrough13.csv', index=False)

In [65]:
# # Built pipline
# # The pipeline's goal is to combine numerous processes that can be cross-validated while modifying various parameters.
# # It does this by allowing set parameters for each step using their names and parameter names separated by a "__"
# # It takes steps as a prameter that contain all the preprocessing, vectorization, and normalization that I need.
# # It saves time by applying any preprocessing to both train and test data without repeating the process.

# pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="char", norm="l2")), ("my_classifier",  XGBClassifier())])

# # define parameter space to test
# params = {
#     "tfidf__ngram_range": [(1, 2), (1, 3)],
#     # tfidf__ngram_range points to TfidfVectorizer -> ngram_range
#     # ngram_range Two words have a higher correlation than the threshold and frequently appear together.
#     # ngram_range=(a,b)-> a is the minimum and b is the maximum size of ngrams
#     "tfidf__max_df": np.arange(0.3, 0.8),
#     # max_df ignore terms that have a document frequency strictly higher than the given threshold
#     "tfidf__min_df": np.arange(5, 100),
#     # min_df ignore terms that have a document frequency strictly lower than the given threshold
#     'my_classifier__min_child_weight': [20,40,80],
#     # min_child_weight is a minimal total of the weights in a child.
#     'my_classifier__max_depth':[50,60,70],  
#     # max_depth is a maximum depth of a tree
#     'my_classifier__gamma':[0.5, 1, 1.5, 2, 5],
#     # gamma is a minimum loss that we need it to split tree
#     'my_classifier__colsample_bytree':[0.6, 0.8, 1.0]
# }

# pipe_clf = BayesSearchCV(
#     pipe, params, cv=pds, verbose=1, n_jobs=2, 
#     scoring='roc_auc')

# # here we still use X but the Randomized search model by use validation set 
# # fit the pipeline
# pipe_clf.fit(X, y)

# print('best score {}'.format(pipe_clf.best_score_))
# print('best score {}'.format(pipe_clf.best_params_))

In [66]:
# # run pipe with optimized parameters
# best_params = pipe_clf.best_params_
# pipe.set_params(**best_params).fit(X, y)
# pipe_pred = pipe.predict(X_test)

In [67]:
# # Use this cell to write the result in the excel sheet.
# submission = pd.DataFrame()

# submission['id'] = data_ts['id']

# submission['label'] = pipe.predict_proba(X_test)[:,1]

# submission.to_csv('sample_submission_walkthrough14.csv', index=False)

🌈 What is the difference between Character n-gram and Word n-gram? Which one tends to suffer more from the OOV issue?
A character n-gram is a set of co-occurring characters within a given window. It’s very similar to word n-grams, only that the window size is at the character level. A bag of character n-grams in the fastText case means a word is represented by a sum of its character n-grams1.
Word n-grams are sequences of n words that are used as features in natural language processing tasks such as text classification and sentiment analysis2.
Character n-grams tend to suffer more from the OOV (out-of-vocabulary) issue because there are many more possible character combinations than word combinations

🌈 What is the difference between stop word removal and stemming? Are these techniques language-dependent?
Stop word removal and stemming are two common techniques used in natural language processing (NLP) to preprocess text data before applying machine learning algorithms. Stop word removal is the process of removing commonly used words such as “the”, “a”, “an”, “in”, “on”, etc., which do not add much meaning to the text and can be safely removed without affecting the overall meaning of the text. Stemming, on the other hand, is the process of reducing words to their base or root form by removing suffixes and prefixes. For example, “running” and “runner” would both be reduced to “run” using stemming.
Stop word removal and stemming are not language-dependent techniques. However, their effectiveness may vary depending on the language being used. For example, stop word removal may be more effective in English than in Chinese because English has a larger number of stop words. Similarly, stemming may be more effective in languages with more regular inflectional morphology such as Spanish than in languages with more irregular morphology such as English.

🌈 Is tokenization techniques language dependent? Why?
Tokenization is a common technique used in natural language processing (NLP) to preprocess text data before applying machine learning algorithms. It involves breaking down a piece of text into smaller units called tokens. These tokens can be words, phrases, or even individual characters.
Tokenization techniques are not language-dependent. However, the effectiveness of tokenization may vary depending on the language being used. For example, tokenization may be more challenging in languages such as Chinese and Japanese because these languages do not use spaces between words

🌈 What is the difference between count vectorizer and tf-idf vectorizer? Would it be feasible to use all possible n-grams? If not, how should you select them?
CountVectorizer and TfidfVectorizer are both techniques used to convert text data into numerical data that can be used in machine learning algorithms. CountVectorizer counts the number of times each word appears in a document and creates a matrix of word counts. TfidfVectorizer, on the other hand, assigns a score to each word based on its frequency in a document and across all documents. This score is called the term frequency-inverse document frequency (tf-idf) score.
TfidfVectorizer is generally considered better than CountVectorizer because it provides more information about the importance of each word. CountVectorizer only focuses on the frequency of words present in the corpus. TfidfVectorizer provides information about how important each word is to a particular document and across all documents.
It would not be feasible to use all possible n-grams because it would result in a very large number of features, which could lead to overfitting. Instead, you should select n-grams based on their relevance to your specific problem. For example, if you are working with text data that contains many technical terms, you may want to include more n-grams that contain those terms